In [ ]:
!pip install transformers[torch] -U
!pip install accelerate -U
!pip install transformers[torch]
!pip install torchinfo
!pip install transformers datasets


In [ ]:
pip install --upgrade torch torchvision


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score,confusion_matrix
from datasets import load_dataset
from transformers import AutoTokenizer
from torchinfo import summary
from transformers import pipeline
from transformers import AutoModelForSequenceClassification,Trainer, TrainingArguments

In [ ]:
df_new=pd.read_csv("data_train_task1_oversampled.csv")
target_map={"surprise":6,"neutral":4,"disgust":1,"joy":3,"anger":0,"fear":2,"sadness":5,"contempt":3 }
df_new['target']=df_new['label'].map(target_map)

In [ ]:
df2=df_new[["text","target"]]
df2.columns=["text","label"]
df2.to_csv("data.csv",index=None)

In [ ]:
from datasets import load_dataset
raw_dataset=load_dataset('csv',data_files="data.csv")

In [ ]:
raw_dataset

In [ ]:
split=raw_dataset['train'].train_test_split(test_size=0.3,seed=7)

In [ ]:
split

In [ ]:
checkpoint='j-hartmann/emotion-english-distilroberta-base'

In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_fn(batch):
     return tokenizer(batch['text'],truncation=True)

In [ ]:
tokenized_datasets=split.map(tokenize_fn,batched=True)

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=7)#num_labels denotes no of classes u want to classify in

In [ ]:
summary(model)  #In Pytorch it does not show output size after each layer as done in Tensor Flow

In [ ]:
training_args = TrainingArguments(
              output_dir='training_dir',
              evaluation_strategy='epoch',
              #save_strategy='epoch'   TO SAVE MODEL AFTER EACH EPOCH AND USE MODEL AFTER WHICH WE GOT BEST ACCURACY BUT REQUIRES LARGE SPACE
              num_train_epochs=3,
              per_device_train_batch_size=16,
              per_device_eval_batch_size=64,
              #LIKE THIS WE HAVE MANY ARGUMENTS WHICH WE CAN CHANGE
)

In [ ]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average='macro')
  return {'accuracy': acc, 'f1': f1}

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model("my_saved_model")

In [ ]:
#storing model directory as zip
import shutil

shutil.make_archive('emo_Model_V1', 'zip', "/content/my_saved_model")
